In [4]:
import os
from google.colab import userdata

Switch Device

In [3]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

API KEYS

In [7]:
from google.colab import userdata
os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

Creating Folder and User Files

In [5]:
user_folder = 'user_documents'

if not os.path.exists(user_folder):
  os.makedirs(user_folder)
  print("Directory Created...")
else:
  print("Directory already exist...")

Directory Created...


In [6]:
from google.colab import files
uploaded = files.upload()

Saving ubuntu-server-guide-2022-08-03.pdf to ubuntu-server-guide-2022-08-03.pdf


In [7]:
for file_name in uploaded:
  os.rename(file_name, f'{user_folder}/{file_name}')
print("Uploaded files moved to user_documents")

Uploaded files moved to user_documents


In [9]:
print("List of files uploaded are: ")
dir = os.listdir(user_folder)

for i in dir :
  if i == '.ipynb_checkpoints':
    dir.pop()
    print("popped: ", i)
    continue
  print(" - ", i)

List of files uploaded are: 
 -  ubuntu-server-guide-2022-08-03.pdf


Document Loader using PyPDFLoader

In [10]:
!pip install -Uq langchain langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [11]:
from langchain_community.document_loaders import PyPDFLoader

In [14]:
!pip install -Uq pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 2.2 MB/s eta 0:00:00


In [15]:
for file in dir:
  file_path = (
    f"{user_folder}/{file}"
  )
  print(file_path)
  loader = PyPDFLoader(file_path)
  pages = loader.load_and_split()

user_documents/ubuntu-server-guide-2022-08-03.pdf


In [16]:
pages[0]

Document(metadata={'source': 'user_documents/ubuntu-server-guide-2022-08-03.pdf', 'page': 0}, page_content='Ubuntu Server Guide\nWelcome to the Ubuntu Server Guide! This site includes information on using Ubuntu Server for Ubuntu 20.04 L TS\n(F ocal F ossa) and later . For an offline version as well as versions for previous releases see below.\nGetting Started:\nBasic Install Install Server Edition using the installer\nAutomated Install Automate installation using autoinstall\nCloud Images Run Ubuntu Server Edition on a public cloud provider\nSoftware Containers Run Ubuntu Server software in dedicated containers\nRefining Y our Infrastructure:\nNetworking Register network devices\nNetworked Storage Configure multiple I/O paths for storage\nSecurity Protect your server and network\nVirtualization Separate your services into virtualized environments\nHigh Availability Cluster systems redundantly to minimize down-time\nMonitoring Ensure infrastructure availability\nConfiguring Y our Servi

In [17]:
for page in pages[0:10]:
  print(page.metadata["page"])
  print(page.page_content)

0
Ubuntu Server Guide
Welcome to the Ubuntu Server Guide! This site includes information on using Ubuntu Server for Ubuntu 20.04 L TS
(F ocal F ossa) and later . For an offline version as well as versions for previous releases see below.
Getting Started:
Basic Install Install Server Edition using the installer
Automated Install Automate installation using autoinstall
Cloud Images Run Ubuntu Server Edition on a public cloud provider
Software Containers Run Ubuntu Server software in dedicated containers
Refining Y our Infrastructure:
Networking Register network devices
Networked Storage Configure multiple I/O paths for storage
Security Protect your server and network
Virtualization Separate your services into virtualized environments
High Availability Cluster systems redundantly to minimize down-time
Monitoring Ensure infrastructure availability
Configuring Y our Services:
Databases Install database servers
Backups Establish your backup plans
Mail Servers Set up e-mail services
Web Serve

Unstructured Documents Loader

In [18]:
!pip install -Uq "langchain-unstructured[local]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.5 MB/s eta 0:00:00
 

In [ ]:
# !pip install -Uq "unstructured[all-docs]"

In [23]:
#No module named 'onnx.defs' docs = loader.load()
!pip uninstall onnx -y
!pip install onnx
!apt-get install -y poppler-utils

Found existing installation: onnx 1.16.2
Uninstalling onnx-1.16.2:
  Successfully uninstalled onnx-1.16.2
  Using cached onnx-1.16.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
Using cached onnx-1.16.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (15.9 MB)


Reading package lists... Done
^C


In [ ]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (24.9 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123625 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [19]:
from langchain_unstructured import UnstructuredLoader
from unstructured.cleaners.core import clean_extra_whitespace

In [21]:
print(f"{user_folder}/{file}")

user_documents/ubuntu-server-guide-2022-08-03.pdf


In [22]:
loader = UnstructuredLoader(
    f"{user_folder}/{file}",
    post_processors=[clean_extra_whitespace],
    mode="elements",
    strategy="fast",
)
loader
docs = loader.load()

PDFInfoNotInstalledError: Unable to get page count. Is poppler installed and in PATH?

In [ ]:
docs[0:10]

Loading Embedding Model using HuggingFaceEmbedding

In [ ]:
!pip install -Uq sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_1 = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-17-2eebfe737e40>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model_1 = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

C_ error solved torch dependences

In [ ]:
# # Reinstall PyTorch
# !pip uninstall -y torch torchvision torchaudio
# !pip install torch torchvision torchaudio

Pinecone Database

In [1]:
!pip install -Uq langchain-pinecone pinecone-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 7.1 MB/s eta 0:00:00


In [2]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
import pinecone

In [8]:
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [9]:
index_name = "book-sync"

if index_name not in pc.list_indexes().names():
  pc.create_index(
      name=index_name,
      dimension=384,
      metric="cosine",
      spec=ServerlessSpec(
          cloud='aws',
          region='us-east-1'
      )
  )
  print("Created index...\n", pc.describe_index(index_name))
else:
  print("Index already exists")

Index already exists


In [10]:
index = pc.Index(index_name)

In [ ]:
# Create instance without doc
docsearch_instance = PineconeVectorStore(
    pinecone_api_key=os.environ["PINECONE_API_KEY"],
    embedding=embedding_model_1,
    index=index,
)

In [ ]:
%%time
# Create instance -> embed doc and upsert
docsearch = PineconeVectorStore.from_documents(
    pinecone_api_key=os.environ["PINECONE_API_KEY"],
    index_name=index_name,
    embedding=embedding_model_1,
    documents=pages,
    batch_size=100,
)

CPU times: user 49 s, sys: 5.05 s, total: 54 s
Wall time: 55.7 s


In [ ]:
# Function to get similar data
def get_similar_doc(query, k=3, score=True):
  if score:
    similar_docs = docsearch.similarity_search_with_score(query, k=k)
  else:
    similar_docs = docsearch.similarity_search(query, k=k)
  return similar_docs

In [ ]:
similar_docs = get_similar_doc("what is web server", k=1)
similar_docs

[(Document(metadata={'page': 277.0, 'source': 'user_documents/ubuntu-server-guide-2022-08-03.pdf'}, page_content='Implementation\nWeb Servers are heavily used in the deployment of Web sites and in this scenario we can use two different implemen-\ntations:\n• Static W eb Server : The content of the server’s response will be the hosted files “as-is”.\n• Dynamic W eb Server : Consist in a Web Server plus an extra software, usually an application server and a\ndatabase. For example, to produce the Web pages you see in the Web browser, the application server might fill\nan HTML template with contents from a database. Due to that we say that the content of the server’s response\nis generated dynamically.\nHTTPD - Apache2 Web Server\nApache is the most commonly used Web server on Linux systems. Web servers are used to serve Web pages requested\nby client computers. Clients typically request and view Web pages using Web browser applications such as Firefox,\nOpera, Chromium, or Internet Explor

*   Chroma DB
*   Chat Function
*   Py file -> integrate to local env











In [11]:
index.delete(delete_all=True)

{}